# Imports

In [3]:
import os  
import glob
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix


import PIL 
import random
import numpy as np
import matplotlib.pyplot as plt 

from tool_preprocessing import *

# Preprocessing

In this notebook, labels are initially considered as categorical.

## Manual Part

If the images are organized in the folders of each label, the following flag must be True

Grande Osmar, deixa a flag como True mesmo

In [4]:
flag_folder_sep = True

In [5]:
if flag_folder_sep:
    base_path = input("Qual é o caminho do dataset?")
    # base_path = 'C:/Users/lucas/OneDrive - unb.br/Documents/UnB/Semestres-ENE/TCC/COVID_Dataset_original'
    results_path = input("Qual será o caminho para os resultados?")
    # results_path = f"C:/Users/Lucas/medical_images_models/results_COVID"
else:
    # base_path = 'C:/Users/lucas/OneDrive - unb.br/Documents/UnB/Semestres-ENE/TCC/The HAM10000 dataset'
    # results_path = f"C:/Users/Lucas/medical_images_models/results_HAM"
    base_path = input("Qual é o caminho do dataset?")
    results_path = input("Qual será o caminho para os resultados?")


In [6]:
if flag_folder_sep :
    label_column = 'label'
    train_df, test_df, val_df  = make_dataset_by_folder(base_path=base_path, label_column=label_column)

else:
    
    path_train_df = f'{base_path}/HAM10000_metadata'
    path_test_df = f'{base_path}/test.csv'
    
    path_train = f"{base_path}/treino"
    path_test = f"{base_path}/test"
    
    paths_image = [path_train, path_test]
    paths_df = [path_train_df, path_test_df]
    label_column = 'dx'
    
    train_df, test_df, val_df = make_dataset_by_df(paths_image, paths_df, label_column=label_column)
    
    

nv - 671
mel - 112
bkl - 110
bcc - 52
akiec - 33
vasc - 15
df - 12


## Analysis

### Train

In [7]:
train_df = check_images_existence(train_df, path_column='path')

In [10]:
train_df.to_csv("teste.csv")

In [8]:
image_analysis_train = image_analysis(train_df)

KeyboardInterrupt: 

In [ ]:
dict_train_qntd = get_label_counts_and_print(train_df, label_column=label_column)
shapes_train = analyze_image_shapes(train_df, min_shape=(800, 800), path_column='path')

Total number of images: 9010
Number of unique labels: 7
Label 'nv' has 6034 images.
Label 'mel' has 1001 images.
Label 'bkl' has 989 images.
Label 'bcc' has 462 images.
Label 'akiec' has 294 images.
Label 'vasc' has 127 images.
Label 'df' has 103 images.
Average image shape - Height: 450.0, Width: 600.0
Number of images with shape smaller than (800, 800): 9010


In [ ]:
dict_train_qntd

{'nv': 6034,
 'mel': 1001,
 'bkl': 989,
 'bcc': 462,
 'akiec': 294,
 'vasc': 127,
 'df': 103}

### Test

In [ ]:
test_df = check_images_existence(test_df, path_column='path')

In [ ]:
image_analysis_test = image_analysis(test_df)

Smallest pixel value: 0
Largest pixel value: 255
Total images processed: 1511
Channel Statistics:
Channel 'R':
  - Average: 193.96235187636344
  - Standard Deviation: 24.606448726550262
Channel 'G':
  - Average: 141.7550379243572
  - Standard Deviation: 31.9625774054364
Channel 'B':
  - Average: 147.87214814814814
  - Standard Deviation: 35.78649271231254


In [ ]:
dict_test_qntd = get_label_counts_and_print(test_df, label_column=label_column)
shapes_test = analyze_image_shapes(test_df, min_shape=(300, 300), path_column='path')

### Validation

In [ ]:
val_df = check_images_existence(val_df, path_column='path')

In [ ]:
image_analysis_val = image_analysis(val_df)

In [ ]:
dict_val_qntd = get_label_counts_and_print(val_df, label_column=label_column)
shapes_val = analyze_image_shapes(val_df, min_shape=(461, 601), path_column='path')

## Model Preparation

In [ ]:
from model_preprocessing import *

Passar de categorial para binário 

Pesos para a loss

### Categorial to number

In [ ]:
labels_dict = labels2dict(train_df, label_column)
labels_dict

In [ ]:
train_label, test_label, val_label = dflabel2number([train_df, test_df, val_df], labels_dict, label_column)

### Weights

In [ ]:
if len(labels_dict) == 1:
    weights = calculate_weights(train_df, labels_dict, dict_train_qntd)
    weights = max(weights)
else:
    weights = calculate_weights(train_df, labels_dict, dict_train_qntd)
    print(weights)

# Model

In [ ]:
from models import *
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms

## Dataset Class

In [ ]:
class CT_Dataset(Dataset):
    def __init__(self, img_path, img_labels, channels, img_transforms=None):
        self.img_path = img_path
        self.img_labels = torch.Tensor(img_labels)
        if channels == 1:
            self.transforms = transforms.Compose([transforms.Grayscale(),
                                                #   transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        elif channels == 3:
            self.transforms = transforms.Compose([#transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        else:
            self.transforms = img_transforms
    
    def __getitem__(self, index):
        # load image
        cur_path = self.img_path[index]
        cur_img = PIL.Image.open(cur_path).convert('RGB')
        cur_img = self.transforms(cur_img)

        return cur_img, self.img_labels[index]
    
    def __len__(self):
        return len(self.img_path)

## GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

In [ ]:
print("Current GPU memory usage:", torch.cuda.memory_allocated() / (1024 ** 2), "MB")
print("Max GPU memory usage:", torch.cuda.max_memory_allocated() / (1024 ** 2), "MB")

torch.cuda.empty_cache()

## Random Seed

In [ ]:
random_seed = 124
np.random.seed(random_seed)

torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

## Training

In [ ]:
try:
    mean_R = image_analysis_val['channel_statistics']['R']['average']
    mean_G = image_analysis_val['channel_statistics']['G']['average']
    mean_B = image_analysis_val['channel_statistics']['B']['average']
    channels = 1 if mean_R == mean_G == mean_B else 3
    
except KeyError:
    channels = image_analysis_val['channels']

In [ ]:
train_dataset = CT_Dataset(img_path=np.array(train_df['path']), img_labels=np.array(train_label), channels=channels)
val_dataset = CT_Dataset(img_path=np.array(val_df['path']), img_labels=np.array(val_label), channels=channels)
test_dataset = CT_Dataset(img_path=np.array(test_df['path']), img_labels=np.array(test_label), channels=channels)

In [ ]:
from trainer import *

In [ ]:
batch_size = 4
Epochs = 20



# model_kernel = VGG16(num_classes=len(labels_dict), input_channels=channels)
# model_kernel = ResNet50(num_classes=len(labels_dict), input_channels=channels)
# model_kernel = ResNet101(num_classes=len(labels_dict), input_channels=channels)
# model_kernel = EfficientNetB0(num_classes=len(labels_dict), input_channels=channels)
# model_kernel = EfficientNetB4(num_classes=len(labels_dict), input_channels=channels)
model_kernel = EfficientNetB7(num_classes=len(labels_dict), input_channels=channels)



trainer = ModelTrainer(model_kernel, device, weights, labels_dict, train_dataset, val_dataset, test_dataset, batch_size= batch_size, epochs=Epochs)

In [ ]:
trainer.loader()
trainer.loss_function()
trainer.optimizer_step()
print("Training Start:")
for epoch in range(Epochs):
    trainer.model.train()

    trainer.train_loss = 0
    trainer.train_acc = 0

    trainer.train()
    trainer.validate()
    history = trainer.loss_acc()


    print(f"Epoch:{epoch + 1} / {Epochs}, lr: {trainer.optimizer.param_groups[0]['lr']:.5f} train loss:{trainer.train_loss:.5f}, train acc: {trainer.train_acc:.5f}, valid loss:{trainer.val_loss:.5f}, valid acc:{trainer.val_acc:.5f}")
        
    # Update the best model if validation loss is the lowest so far
    if trainer.val_loss < trainer.best_val_loss:
        trainer.best_val_loss = trainer.val_loss
        trainer.best_model_state = trainer.model.state_dict()

    print(f'The best val loss is {trainer.best_val_loss}.\n')
    
    # Load the best model state
    if trainer.best_model_state is not None:
        trainer.model.load_state_dict(trainer.best_model_state)
    model = trainer.model
    
trainer.test()
metrics_df = trainer.metrics()

In [ ]:
metrics_df = trainer.metrics()
metrics_df = metrics_df.applymap(lambda x: str(x).replace('.', ','))

# Metrics

In [ ]:
from model_metrics import *                                                                         

In [ ]:

# if flag_folder_sep:
#     results_path = f"C:/Users/Lucas/medical_images_models/results_COVID/Model_{model.get_name()}__Epoch_{Epochs}__Batch_{batch_size}__Accuracy_{metrics_df['Accuracy'][0]}"

# else:
#     results_path = f"C:/Users/Lucas/medical_images_models/results_HAM/Model_{model.get_name()}__Epoch_{Epochs}__Batch_{batch_size}__Accuracy_{metrics_df['Accuracy'][0]}"

In [ ]:
metrics_df

In [ ]:
metrics_df.to_csv(f"{results_path}/Model_{model.get_name()}__Epoch_{Epochs}__Batch_{batch_size}__Accuracy_{metrics_df['Accuracy'][0]}.csv", index=False)

In [ ]:
plot_metrics(history, path=results_path)

## Plot Images - True Predicted

In [ ]:
inverted_labels_dict = {value: key for key, value in labels_dict.items()}
inverted_labels_dict

In [ ]:
plot_image_pred_true(model, test_dataset, device, inverted_labels_dict, num_images_to_plot=20, plot_images=True)

# Save Model

In [ ]:
torch.save(model.state_dict(), f"{results_path}/Model_{model.get_name()}__Epoch_{Epochs}__Batch_{batch_size}__Accuracy_{metrics_df['Accuracy'][0]}.pth")